# For certain metrics, aggregate them to quarterly in `merge_data`

* Tiffany's Teams message: 
 > you can do it within merge_data for now, it'll be the quickest way to get your quarterly averages. 
merge_data is simply concatenating schedule, summary speeds, rt vs schedule by route-direction-peak/offpeak-single day together. You can take that and aggregate several days into quarterly.
Here's the script for average_segment_speeds that mocks up aggregating across single days / using weighted averages, and you'll see how some of the functions are put together, specifically tagging the quarter-year / weekday or weekend / weighted averages (using n_trips) to weight the metric column. https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/scripts/average_segment_speeds.py
* Metrics to Roll Up
    * Avg scheduled minute
    * VP per minute
    * Spatial Accuracy

In [1]:
import _report_utils
import _section1_utils as section1
import _section2_utils as section2
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, segment_calcs
from segment_speed_utils.project_vars import COMPILED_CACHED_VIEWS, PROJECT_CRS
from shared_utils import (
    catalog_utils,
    portfolio_utils,
    rt_dates,
    rt_utils,
    time_helpers,
)
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

# Data Dictionary
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_date_list = [rt_dates.DATES["feb2025"]]

In [4]:
analysis_date = rt_dates.DATES["feb2025"]

In [5]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

In [6]:
sdi = "San Diego Metropolitan Transit System"

In [7]:
df = section2.load_schedule_vp_metrics(sdi)

In [8]:
df.columns

Index(['schedule_gtfs_dataset_key', 'dir_0_1', 'Period',
       'Average Scheduled Service (trip minutes)',
       'Average Stop Distance (miles)', '# scheduled trips', 'Trips per Hour',
       'is_express', 'is_rapid', 'is_rail', 'is_coverage', 'is_downtown_local',
       'is_local', 'Date', 'Route typology', '# Minutes with 1+ VP per Minute',
       '# Minutes with 2+ VP per Minute', 'Aggregate Actual Service Minutes',
       'Aggregate Scheduled Service Minutes (all trips)', '# VP',
       '# VP within Scheduled Shape', '# Early Arrival Trips',
       '# On-Time Trips', '# Late Trips', '# Trips with VP',
       'Average VP per Minute', '% VP within Scheduled Shape',
       'pct_rt_journey_atleast1_vp', 'pct_rt_journey_atleast2_vp',
       '% Scheduled Trip w/ 1+ VP/Minute', '% Scheduled Trip w/ 2+ VP/Minute',
       'Realtime versus Scheduled Service Ratio',
       'Average Actual Service (Trip Minutes)', 'GTFS Availability',
       'Speed (MPH)', 'route_long_name', 'route_short_nam

## Subset df

In [9]:
df2 = df[
    [
        "Date",
        "Organization",
        "route_long_name",
        "route_short_name",
        "Route",
        "Route ID",
        "Trips per Hour",
        "dir_0_1",
        "Direction",
        "# scheduled trips",
        "# Trips with VP",
        "Period",
        "Average Scheduled Service (trip minutes)",
        "Average VP per Minute",
        "% VP within Scheduled Shape",
        "ruler_for_vp_per_min",
        "ruler_100_pct"
    ]
]

In [10]:
# Filter out to all day
all_day = df2.loc[df2.Period == "all_day"].reset_index(drop=True)

In [11]:
all_day.sample()

,Date,Organization,route_long_name,route_short_name,Route,Route ID,Trips per Hour,dir_0_1,Direction,# scheduled trips,# Trips with VP,Period,Average Scheduled Service (trip minutes),Average VP per Minute,% VP within Scheduled Shape,ruler_for_vp_per_min,ruler_100_pct
115,2023-07-12,San Diego Metropolitan Transit System,Downtown - Rancho Bernardo Transit Station,20,20 Downtown - Rancho Bernardo Transit Station,20,1.83,0.00,Northbound,44,43,all_day,63.59,1.51,92.00,2,100


### The results are really off for `Average VP per Minute`

In [13]:
def quarterly_rollup(all_day:pd.DataFrame)->pd.DataFrame:
    """
    Using the dataframe that houses only all_day values,
    roll up months to each quarter for certain metrics.
    """
    # Turn date to quarters
    all_day["quarter"] = pd.PeriodIndex(all_day.Date, freq="Q").astype("str")

    quarterly_metrics = segment_calcs.calculate_weighted_averages(
    
    df=all_day,
    group_cols=[
        "quarter",
        "Organization",
        "Route",
        "dir_0_1",
        "Direction",
    ],
    metric_cols=[
        "Average VP per Minute",
        "% VP within Scheduled Shape",
        "Average Scheduled Service (trip minutes)",
        "ruler_100_pct",
        "ruler_for_vp_per_min"
    ],
    weight_col="# Trips with VP",
    )
    return quarterly_metrics 

In [14]:
all_metrics_test = quarterly_rollup(all_day)

## Test charts

In [22]:
import yaml

with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)

In [23]:
# Color Palette
with open("color_palettes.yml") as f:
    color_dict = yaml.safe_load(f)

In [24]:
all_metrics_test.sample(3)

,quarter,Organization,Route,Route ID,dir_0_1,Direction,Average VP per Minute,% VP within Scheduled Shape,Average Scheduled Service (trip minutes),ruler_100_pct,ruler_for_vp_per_min,# Trips with VP
1596,2025Q1,San Diego Metropolitan Transit System,909 Otay Mesa Transit Center - SWC at Otay Mesa,909,1.00,Westbound,2.08,94.00,25.07,100.00,2.00,30
1399,2024Q4,San Diego Metropolitan Transit System,872 El Cajon Shuttle Loop Counterclockwise,872,1.00,Northbound,2.13,98.67,19.54,100.00,2.00,78
1141,2024Q3,San Diego Metropolitan Transit System,227 Imperial Beach - Otay Mesa TC,227,1.00,Westbound,2.18,92.33,30.47,100.00,2.00,285


In [25]:
all_metrics_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1645 entries, 0 to 1644
Data columns (total 12 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   quarter                                   1645 non-null   object 
 1   Organization                              1645 non-null   object 
 2   Route                                     1645 non-null   object 
 3   Route ID                                  1645 non-null   object 
 4   dir_0_1                                   1645 non-null   float64
 5   Direction                                 1645 non-null   object 
 6   Average VP per Minute                     1645 non-null   float64
 7   % VP within Scheduled Shape               1645 non-null   float64
 8   Average Scheduled Service (trip minutes)  1645 non-null   float64
 9   ruler_100_pct                             1645 non-null   float64
 10  ruler_for_vp_per_min                

In [26]:
# Charts
import altair as alt

alt.data_transformers.enable("default", max_rows=None)

DataTransformerRegistry.enable('default')

In [27]:
def grouped_bar_chart(
    df: pd.DataFrame,
    color_col: str,
    y_col: str,
    offset_col: str,
    title: str,
    subtitle: str,
    range_color: list,
    quarter: bool = False,
) -> alt.Chart:

    tooltip_cols = [
        "Route",
        "Direction",
        color_col,
        y_col,
    ]

    if quarter == False:
        # Clean dataframe
        tooltip_cols.append("Period")
        tooltip_cols.append("Date")
        df = clean_data_charts(df, y_col)

        chart = (
            alt.Chart(df)
            .mark_bar(size=5)
            .encode(
                x=alt.X(
                    "yearmonthdate(Date):O",
                    title=["Date"],
                    axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                ),
                y=alt.Y(f"{y_col}:Q", title=_report_utils.labeling(y_col)),
                xOffset=alt.X(
                    f"{offset_col}:N", title=_report_utils.labeling(offset_col)
                ),
                color=alt.Color(
                    f"{color_col}:N",
                    title=_report_utils.labeling(color_col),
                    scale=alt.Scale(range=range_color),
                ),
                tooltip=tooltip_cols,
            )
        )
    else:
        tooltip_cols.append("quarter")
        chart = (
            alt.Chart(df)
            .mark_bar(size=5)
            .encode(
                x=alt.X(
                    "quarter",
                    title=["Quarter"],
                    axis=alt.Axis(labelAngle=-45),
                ),
                y=alt.Y(f"{y_col}:Q", title=_report_utils.labeling(y_col)),
                xOffset=alt.X(
                    f"{offset_col}:N", title=_report_utils.labeling(offset_col)
                ),
                color=alt.Color(
                    f"{color_col}:N",
                    title=_report_utils.labeling(color_col),
                    scale=alt.Scale(range=range_color),
                ),
                tooltip=tooltip_cols,
            )
        )

    chart = (chart).properties(
        title={
            "text": title,
            "subtitle": [subtitle],
        },
        width=400,
        height=250,
    )

    return chart

In [28]:
all_metrics_test.columns

Index(['quarter', 'Organization', 'Route', 'Route ID', 'dir_0_1', 'Direction',
       'Average VP per Minute', '% VP within Scheduled Shape',
       'Average Scheduled Service (trip minutes)', 'ruler_100_pct',
       'ruler_for_vp_per_min', '# Trips with VP'],
      dtype='object')

In [29]:
# Create dropdown
routes_list = all_metrics_test["Route"].unique().tolist()

route_dropdown = alt.binding_select(
    options=routes_list,
    name="Routes: ",
)
# Column that controls the bar charts
xcol_param = alt.selection_point(
    fields=["Route"], value=routes_list[0], bind=route_dropdown
)

### Average Scheduled Service (trip minutes)

In [30]:
(
    grouped_bar_chart(
        df=all_metrics_test,
        color_col="Direction",
        y_col="Average Scheduled Service (trip minutes)",
        offset_col="Direction",
        title=readable_dict["avg_scheduled_min_graph"]["title"],
        subtitle=readable_dict["avg_scheduled_min_graph"]["subtitle"],
        range_color=color_dict["four_colors"],
        quarter=True,
    )
).add_params(xcol_param).transform_filter(xcol_param)

alt.Chart(...)

### % VP within Scheduled Shape

In [31]:
def base_facet_with_ruler_chart(
    df: pd.DataFrame,
    y_col: str,
    ruler_col: str,
    title: str,
    subtitle: str,
    domain_color: list,
    range_color: list,
    quarter: bool = False,
) -> alt.Chart:

    tooltip_cols = [
        "Route",
        "Direction",
        y_col,
    ]

    # Set y-axis
    max_y = section2.set_y_axis(df, y_col)

    # Clean dataframe
    df = section2.clean_data_charts(df, y_col)

    # Create color scale
    color_scale = alt.Scale(domain=domain_color, range=range_color)

    # Create ruler
    ruler = (
        alt.Chart(df)
        .mark_rule(color="red", strokeDash=[10, 7])
        .encode(y=f"mean({ruler_col}):Q")
    )
    if quarter == False:
        tooltip_cols.append("Period")
        tooltip_cols.append("Date")
        chart = (
            alt.Chart(df)
            .mark_bar(size=7, clip=True)
            .encode(
                x=alt.X(
                    "yearmonthdate(Date):O",
                    title=[x_col],
                    axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                ),
                y=alt.Y(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(domain=[0, max_y]),
                ),
                color=alt.Color(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=color_scale,
                ),
                tooltip=df[tooltip_cols].columns.tolist(),
            )
        )
    else:
        tooltip_cols.append("quarter")
        chart = (
            alt.Chart(df)
            .mark_bar(size=7, clip=True)
            .encode(
                x=alt.X(
                    "quarter",
                    title="Quarter",
                    axis=alt.Axis(labelAngle=-45),
                ),
                y=alt.Y(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=alt.Scale(domain=[0, max_y]),
                ),
                color=alt.Color(
                    f"{y_col}:Q",
                    title=_report_utils.labeling(y_col),
                    scale=color_scale,
                ),
                tooltip=df[tooltip_cols].columns.tolist(),
            )
        )
    # All charts
    chart = (chart + ruler).properties(width=200, height=250)
    chart = chart.facet(
        column=alt.Column(
            "Direction:N",
        )
    ).properties(
        title={
            "text": title,
            "subtitle": [subtitle],
        }
    )

    return chart

In [32]:
color_dict["spatial_accuracy_range"]

['#dd217d', '#fc5c04', '#ff9c42', '#fcb40e', '#e9d868', '#ccbb44']

In [33]:
base_facet_with_ruler_chart(
    df=all_metrics_test,
    y_col="% VP within Scheduled Shape",
    ruler_col="ruler_100_pct",
    title=readable_dict["spatial_accuracy_graph"]["title"],
    subtitle=readable_dict["spatial_accuracy_graph"]["subtitle"],
    domain_color=color_dict["spatial_accuracy_domain"],
    range_color=color_dict["spatial_accuracy_range"],
    quarter=True,
).add_params(xcol_param).transform_filter(xcol_param)

alt.FacetChart(...)

### Average VP per Minute

In [34]:
(
    (
        base_facet_with_ruler_chart(
            all_metrics_test,
            "Average VP per Minute",
            "ruler_for_vp_per_min",
            readable_dict["vp_per_min_graph"]["title"],
            readable_dict["vp_per_min_graph"]["subtitle"],
            color_dict["vp_domain"],
            color_dict["vp_range"],
            quarter=True,
        )
    )
    .add_params(xcol_param)
    .transform_filter(xcol_param)
)

alt.FacetChart(...)

In [35]:
all_metrics_test.loc[
    (all_metrics_test.Organization == "San Diego Metropolitan Transit System")
    & (all_metrics_test["Route"] == '1 Fashion Valley - La Mesa')
    & (all_metrics_test.quarter == "2023Q3")
]

,quarter,Organization,Route,Route ID,dir_0_1,Direction,Average VP per Minute,% VP within Scheduled Shape,Average Scheduled Service (trip minutes),ruler_100_pct,ruler_for_vp_per_min,# Trips with VP,% VP within Scheduled Shape_str,Average VP per Minute_str
368,2023Q3,San Diego Metropolitan Transit System,1 Fashion Valley - La Mesa,1,0.00,Eastbound,1,97,62.04,100.00,2.00,150,97,1
369,2023Q3,San Diego Metropolitan Transit System,1 Fashion Valley - La Mesa,1,1.00,Westbound,1,94,57.56,100.00,2.00,156,94,1


In [36]:
stop

NameError: name 'stop' is not defined

## Turn off group keys for the `segment_calcs.calculate_weighted_averages` function
* This didn't work at all. 

In [ ]:
def calculate_weighted_averages(
    df: pd.DataFrame, group_cols: list, metric_cols: list, weight_col: str
):
    """
    For certain aggregations, we need to calculate a weighted average,
    weighted by the number of trips.

    If we want peak/offpeak weighted calculations,
    we can take time-of-day (AM peak, PM peak) and
    get a peak speed calculation, after weighting by the number
    of trips present in each time-of-day bin.

    Ex: metric_cols = ['p20_mph', 'p50_mph', 'p80_mph']
    weight_cols = 'n_trips'

    """
    for c in metric_cols:
        df[c] = df[c] * df[weight_col]

    df2 = (
        df.groupby(group_cols)
        .agg({c: "sum" for c in metric_cols + [weight_col]})
        .reset_index()
    )

    for c in metric_cols:
        df2[c] = df2[c].divide(df2[weight_col]).round(2)

    return df2

In [ ]:
all_metrics_test2 = calculate_weighted_averages(
    df=all_day,
    group_cols=[
        "quarter",
        "Organization",
        "schedule_gtfs_dataset_key",
        "Route ID",
        "dir_0_1",
        "Direction",
    ],
    metric_cols=[
        "Average VP per Minute",
        "% VP within Scheduled Shape",
        "Average Scheduled Service (trip minutes)",
    ],
    weight_col="# Trips with VP",
)

In [ ]:
all_metrics_test2.loc[
    (all_metrics_test2.Organization == "San Diego Metropolitan Transit System")
    & (all_metrics_test2["Route ID"] == "1")
    & (all_metrics_test2.quarter == "2023Q2")
    & (all_metrics_test2.Direction == "Eastbound")
]